In [1]:
import pandas as pd
from neten.generate_network import NetWorkEnhancement
from neten.leidenalg_clustering import LeidenAlg

In [2]:
#parameters for Leiden
optimization_method = recursive_method = {
    'method': 'Surprise',
    'partition_type': 'SurpriseVertexPartition',
    'weights': True,
    'signed': False,
    'resolution_parameter': None,
    'n_iterations': -1,
    'multi_resolution': False 
}
max_size = 100  # Maximum allowable size of a module.

#parameters for Network enhancement
k = None  # number of neighbors, if NULL then defaults to ceil(20,ncol(data))
alpha = 0.9  #regularization parameter
diffusion = 2  # diffusion parameter, typical values are 0.5, 1.0, 1.2, 2.0


In [3]:
datadir = "./data/"
name = "swip"
df = pd.read_csv(f"{datadir}{name}_tmt.csv")


In [4]:
# input should be TMT data
dm = df.pivot_table(values="Abundance", index=[
    "Protein"], columns=["Mixture", "BioFraction", "Genotype"])
# drop NA
print(f'{dm.isna().sum().sum()} proteins with any missing values are removed.')
dm = dm.dropna()
# make  pearson correlation matrix
corr = dm.T.corr(method="pearson")


0 proteins with any missing values are removed.


In [5]:
neten = NetWorkEnhancement(corr,k=None,alpha=alpha,diffusion=diffusion)
df_corr = corr.to_csv(f"{datadir}{name}_corr.csv")
print(f"saved: {datadir}{name}_corr.csv")


saved: ./data/swip_corr.csv


In [6]:
ne_adjm=neten.Enhance()
df_adjm = ne_adjm.to_csv(f"{datadir}{name}_adjm.csv")
print(f"saved: {datadir}{name}_adjm.csv")


Generating Network of 5897 proteins...
Building dominantset...
Building TransitionField...
Eigenvalue Decomposing...
Calculating weight...
I appreciate to your patience
Almost done...
saved: ./data/swip_adjm.csv


In [ ]:
Leiden = LeidenAlg(parameters=optimization_method, recursive=True,
                   recursive_parameters=recursive_method, max_size=max_size)
community = Leiden.Run(df_adjm)
community.to_csv(f"{datadir}{name}_partition.csv")
print(f"saved: {datadir}{name}_partition.csv")


In [ ]:
postprocessed = Leiden.postprocess()
postprocessed.to_csv(f"{datadir}{name}_postprocessed.csv")
print(f"saved: {datadir}{name}_postprocessed.csv")